<h1> Deploying and predicting with model </h1>

This notebook illustrates:
<ol>
<li> Deploying model
<li> Predicting with model
</ol>

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/GCP-trainings

In [2]:
# Ensure the right version of Tensorflow is installed.
!pip freeze | grep tensorflow==2.1

In [4]:
# change these to try this notebook out
BUCKET = 'trainings-ml-deployment-ag-2'
PROJECT = 'rare-result-248415'
REGION = 'europe-west1'

In [5]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'  

<h2> Deploy trained model </h2>
<p>
Deploying the trained model to act as a REST web service is a simple gcloud call.

In [6]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model/exporter/

gs://trainings-ml-deployment-ag-2/babyweight/trained_model/exporter/
gs://trainings-ml-deployment-ag-2/babyweight/trained_model/exporter/1638522169/
gs://trainings-ml-deployment-ag-2/babyweight/trained_model/exporter/1638522172/
gs://trainings-ml-deployment-ag-2/babyweight/trained_model/exporter/temp-b'1638522169'/


In [7]:
%%bash
MODEL_NAME="babyweight_ag_new_2"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/babyweight/trained_model/exporter/ | tail -1)

gcloud ai-platform models create ${MODEL_NAME} --regions='europe-west1'

Using endpoint [https://ml.googleapis.com/]
Created ai platform model [projects/rare-result-248415/models/babyweight_ag_new_2].


In [8]:
%%bash
MODEL_NAME="babyweight_ag_new_2"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/babyweight/trained_model/exporter/ | tail -1)
gcloud ai-platform versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Using endpoint [https://us-central1-ml.googleapis.com/]
ERROR: (gcloud.ai-platform.versions.create) NOT_FOUND: Field: parent Error: The model resource: (689107243229, babyweight_ag_new_2) was not found. Please create the model resource first by using 'gcloud ai-platform models create babyweight_ag_new_2'.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: "The model resource: (689107243229, babyweight_ag_new_2) was not\
      \ found. Please create the model resource first by using 'gcloud ai-platform\
      \ models create babyweight_ag_new_2'."
    field: parent


CalledProcessError: Command 'b'MODEL_NAME="babyweight_ag_new_2"\nMODEL_VERSION="v1"\nMODEL_LOCATION=$(gsutil ls gs://${BUCKET}/babyweight/trained_model/exporter/ | tail -1)\ngcloud ai-platform versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION\n'' returned non-zero exit status 1.

<h2> Use model to predict (online prediction) </h2>
<p>
Send a JSON request to the endpoint of the service to make it predict a baby's weight. The order of the responses are the order of the instances.

In [14]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = 'babyweight_ag'
MODEL_VERSION = 'v1'

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
    {
      'key': 'b1',
      'is_male': 'True',
      'mother_age': 23.0,
      'plurality': 'Single(1)',
      'gestation_weeks': 40
    }
  ]
}

    
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{"predictions": [{"predictions": [2.3173141479492188]}]}'


<h2> Use model to predict (batch prediction) </h2>
<p>
Batch prediction is commonly used when you thousands to millions of predictions.
Create a file with one instance per line and submit using gcloud.

In [15]:
%%writefile inputs.json
{"key": "b1", "is_male": "True", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}
{"key": "g1", "is_male": "False", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}

Writing inputs.json


In [16]:
%%bash
INPUT=gs://${BUCKET}/babyweight/batchpred/inputs.json
OUTPUT=gs://${BUCKET}/babyweight/batchpred/outputs
gsutil cp inputs.json $INPUT
gsutil -m rm -rf $OUTPUT 
gcloud ai-platform jobs submit prediction babypred_$(date -u +%y%m%d_%H%M%S) \
  --data-format=TEXT --region ${REGION} \
  --input-paths=$INPUT \
  --output-path=$OUTPUT \
  --model=babyweight --version=ml_on_gcp

jobId: babypred_200629_173121
state: QUEUED


Copying file://inputs.json [Content-Type=application/json]...
/ [1 files][  205.0 B/  205.0 B]                                                
Operation completed over 1 objects/205.0 B.                                      
CommandException: 1 files/objects could not be removed.
Job [babypred_200629_173121] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe babypred_200629_173121

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs babypred_200629_173121


Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License